In [1]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10936751051029595696
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4183621632
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2932910418668083976
physical_device_desc: "device: 0, name: GeForce GTX 1660 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [16]:
import os 
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
seed=101
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
            directory='corona_tweets/train',
            labels='inferred',
            seed=seed,
            subset='training',
            validation_split=0.2)

Found 40994 files belonging to 5 classes.
Using 32796 files for training.


In [3]:
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
            directory='corona_tweets/train',
            labels='inferred',
            seed=seed,
            subset='validation',
            validation_split=0.2)

Found 40994 files belonging to 5 classes.
Using 8198 files for validation.


In [4]:
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization()
#adapt sets the layers vocabulary
encoder.adapt(train_ds.map(lambda text, label: text))

In [5]:
vocab = np.array(encoder.get_vocabulary())
print(vocab[:20])

['' '[UNK]' 'covid' 'prices' 'food' 'store' 'supermarket' 'grocery'
 'people' 'amp' 'consumer' 'shopping' 'online' 'get' 'need' 'pandemic'
 'us' 'workers' 'panic' 'like']


In [6]:
model = tf.keras.Sequential([
    #encoder conversts text to tokens of sequence indices
    encoder,
    #converts series of words to a series of vector embeddings
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=512,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.LSTM(512, return_sequences=True),
    tf.keras.layers.LSTM(256, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5)
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 512)         16961536  
_________________________________________________________________
lstm (LSTM)                  (None, None, 512)         2099200   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 256)         787456    
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         197120    
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 64)          49408     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0

In [7]:
epochs = 6
loss = tf.keras.losses.SparseCategoricalCrossentropy(name='sparse_categorical_crossentropy', from_logits=True)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss=loss, metrics=['accuracy'])

In [8]:
history = model.fit(x=train_ds,
                    validation_data=val_ds,
                    epochs=epochs)

Epoch 1/6
1025/1025 [==============================] - 70s 53ms/step - loss: 1.0843 - accuracy: 0.5503 - val_loss: 0.7564 - val_accuracy: 0.7203
Epoch 2/6
1025/1025 [==============================] - 52s 51ms/step - loss: 0.6435 - accuracy: 0.7706 - val_loss: 0.7462 - val_accuracy: 0.7280
Epoch 3/6
1025/1025 [==============================] - 49s 48ms/step - loss: 0.4438 - accuracy: 0.8478 - val_loss: 0.8166 - val_accuracy: 0.7308
Epoch 4/6
1025/1025 [==============================] - 48s 47ms/step - loss: 0.3252 - accuracy: 0.8936 - val_loss: 0.9349 - val_accuracy: 0.7251
Epoch 5/6
1025/1025 [==============================] - 48s 47ms/step - loss: 0.2483 - accuracy: 0.9191 - val_loss: 0.9607 - val_accuracy: 0.7310
Epoch 6/6
1025/1025 [==============================] - 48s 47ms/step - loss: 0.1994 - accuracy: 0.9348 - val_loss: 1.0868 - val_accuracy: 0.7277


In [12]:
import pickle
outfile = open('history_lstm', 'wb')
pickle.dump(history.history, outfile)
outfile.close()

In [9]:
test_ds = tf.keras.preprocessing.text_dataset_from_directory(
            directory='corona_tweets/test',
            labels='inferred')

Found 40994 files belonging to 5 classes.


In [10]:
model.evaluate(test_ds)

1282/1282 [==============================] - 20s 15ms/step - loss: 0.4139 - accuracy: 0.8808


[0.41391855478286743, 0.880763053894043]

In [ ]:
#1: [0.4081849157810211, 0.8810069561004639]
#2: [0.3951323330402374, 0.8812509179115295]
#3: [0.41391855478286743, 0.880763053894043]

In [12]:
#avereging test loss and accuracy over 3 attempts
ave_result = np.add([0.41391855478286743, 0.880763053894043], np.add([0.4081849157810211, 0.8810069561004639], [0.3951323330402374, 0.8812509179115295]))/3
ave_result

array([0.40574527, 0.88100698])